# Clustering Crypto

In [75]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [76]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path, index_col = [0])
df_crypto.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [77]:
df_crypto['IsTrading'].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [78]:
# Keep all the cryptocurrencies that are being traded.
df_crypto.drop(df_crypto[df_crypto['IsTrading'] == False].index, inplace = True)
df_crypto.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [79]:
# Remove the "IsTrading" column. 
df_crypto = df_crypto.drop(['IsTrading'], axis =1)
df_crypto.head(10)




,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [80]:
for col in df_crypto.columns:
    print(f"Column {col} has {df_crypto[col].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [81]:
# Remove rows that have at least 1 null value.
df_crypto = df_crypto.dropna()


In [82]:
# Keep the rows where coins are mined.
df_crypto.drop(df_crypto[df_crypto['TotalCoinsMined'] <= 0.00].index, inplace = True)
df_crypto.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [83]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_crypto_names = df_crypto[["CoinName"]]
df_crypto_names

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [84]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df_crypto = df_crypto.drop(['CoinName'], axis =1)
df_crypto.head(10)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [85]:
# Use get_dummies() to create variables for text features.
df_crypto_x = pd.get_dummies(df_crypto, columns=['Algorithm','ProofType'])
df_crypto_x.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# Standardize the data with StandardScaler().
df_crypto_x_scaled = StandardScaler().fit_transform(df_crypto_x)
df_crypto_x_scaled


array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [92]:
# Using PCA to reduce dimension to three principal components.
pca =PCA(n_components =3)
crypto_pca = pca.fit_transform(df_crypto_x_scaled)
crypto_pca

array([[-0.36420787,  0.9663873 , -0.5335158 ],
       [-0.34756605,  0.9664756 , -0.53396534],
       [ 2.29653341,  1.64905225, -0.59196635],
       ...,
       [ 0.3309176 , -2.30633566,  0.43143845],
       [-0.04103145, -1.84651266,  0.36053597],
       [-0.30797462,  0.76674459, -0.27472622]])

In [96]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
data = crypto_pca, columns=["PC1", "PC2","PC3"], index=df_crypto.index)
pcs_df.head(10)

,PC1,PC2,PC3
42,-0.364208,0.966387,-0.533516
404,-0.347566,0.966476,-0.533965
1337,2.296533,1.649052,-0.591966
BTC,-0.130537,-1.300086,0.179656
ETH,-0.141350,-1.987428,0.349265
LTC,-0.187471,-1.179268,-0.011490
DASH,-0.388384,1.261976,-0.494569
XMR,-0.125761,-2.158518,0.348797
ETC,-0.139795,-1.987535,0.349250
ZEC,-0.041030,-1.846513,0.360536


In [110]:
pcs_df

,PC1,PC2,PC3
42,-0.364208,0.966387,-0.533516
404,-0.347566,0.966476,-0.533965
1337,2.296533,1.649052,-0.591966
BTC,-0.130537,-1.300086,0.179656
ETH,-0.141350,-1.987428,0.349265
...,...,...,...
ZEPH,2.495855,0.906992,-0.092946
GAP,-0.362256,0.966262,-0.533537
BDX,0.330918,-2.306336,0.431438
ZEN,-0.041031,-1.846513,0.360536


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [98]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1,11))

for i in k:   
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k":k, "inertia" :inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y="inertia", title ="Elbow curve", xticks = k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [99]:
# Initialize the K-Means model.
model = KMeans(n_clusters =4, random_state = 5)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[3 3 3 1 1 1 3 1 1 1 3 1 3 3 1 3 1 1 3 3 1 1 1 1 1 3 1 1 1 3 1 3 1 1 3 3 1
 1 1 1 1 1 3 3 1 1 1 1 1 3 3 1 3 1 1 1 1 3 1 1 3 1 3 3 3 1 1 1 3 3 3 3 3 1
 1 1 3 3 1 3 1 3 3 1 1 1 1 3 3 1 3 1 1 3 3 1 3 3 1 1 3 3 1 3 3 1 3 1 3 1 3
 1 3 3 1 1 3 1 1 1 3 1 1 1 1 1 3 3 1 1 1 3 1 3 1 1 3 1 3 1 3 3 1 1 3 1 1 3
 3 1 3 1 3 3 3 1 1 1 1 3 3 3 3 3 1 1 3 3 3 3 3 1 3 3 3 3 3 1 3 1 3 3 1 3 1
 3 3 1 3 1 3 1 3 1 3 3 3 3 1 3 3 3 3 3 1 1 3 3 1 1 3 3 3 3 3 1 3 3 3 3 3 3
 3 3 1 3 3 3 3 3 3 1 1 1 3 3 3 3 1 3 1 3 3 1 3 1 1 3 1 1 3 1 3 3 3 1 3 3 1
 3 3 3 3 3 3 3 1 3 1 3 3 3 3 1 3 1 3 1 1 1 1 3 1 3 3 1 3 1 1 1 3 1 3 1 1 1
 3 1 3 1 3 3 0 1 3 1 1 1 1 1 3 3 1 3 3 3 1 3 1 3 1 3 1 3 3 3 3 1 3 3 1 3 3
 3 1 1 1 1 3 3 3 3 1 3 1 1 1 3 3 1 1 3 3 1 3 1 1 1 3 1 1 3 3 3 1 1 1 3 3 3
 1 1 3 1 1 1 1 3 0 0 1 1 1 3 0 3 3 3 3 1 1 1 1 3 3 3 1 3 1 3 3 3 3 1 3 3 1
 3 3 1 1 3 1 3 1 1 1 1 3 3 1 3 1 3 3 3 3 3 3 1 1 1 3 3 3 3 3 3 1 3 1 1 1 1
 3 3 3 3 1 3 3 1 3 3 1 0 1 3 1 1 3 3 1 3 1 1 3 1 1 3 1 3 1 3 3 1 3 3 3 3 3
 1 1 1 3 3 3 1 3 1 3 1 3 

In [118]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = df_crypto.merge(pcs_df, left_index=True, right_index=True )

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = df_crypto_names["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.364208,0.966387,-0.533516,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.347566,0.966476,-0.533965,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.296533,1.649052,-0.591966,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.130537,-1.300086,0.179656,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.141350,-1.987428,0.349265,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.187471,-1.179268,-0.011490,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.388384,1.261976,-0.494569,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.125761,-2.158518,0.348797,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.139795,-1.987535,0.349250,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.041030,-1.846513,0.360536,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [128]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(clustered_df, x= "PC1", y="PC2", z="PC3",hover_name="CoinName", hover_data=["Algorithm"],
                    color="Class", 
                    symbol="Class",width=800)
fig.update_layout(legend = dict(x=0, y=1))
fig.show()

In [129]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm','ProofType','TotalCoinSupply', 'TotalCoinsMined','Class'], 
                          sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [133]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df['CoinName'].count()} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [137]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df_scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
clustered_df_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [145]:
clustered_df1["Class"] = clustered_df[["Class"]]
clustered_df1.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,3
404,5.320000e-04,0.001066,404Coin,3
1337,3.141593e-01,0.029576,EliteCoin,3
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,3
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [21]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
clustered_df1 = pd.DataFrame(
data = clustered_df_scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)
pcs_df.head(10)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
clustered_df1["CoinName"] = clustered_df[["CoinName"]]
clustered_df1

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
clustered_df1["Class"] = clustered_df[["Class"]]
clustered_df1

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [146]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

clustered_df1.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', 
legend='top', height=400, width=400, hover_cols=["CoinName"])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)